In [2]:
import pandas as pd
import filecmp

# Загрузка данных

In [52]:
%%bash
less */salami_chords.txt -> songs_full.txt

In [83]:
ds = pd.DataFrame(columns=['title','artist', 'number', 'duration'])
title, artist, number, duration = None, None, None, None
with open('songs.txt') as f:
    for line in f:
        if line.startswith('==>'):
            number = line[4:8]
        elif line.startswith('# title'):
            title = line.split(': ')[1].strip(' \n')
        elif line.startswith('# artist'):
            artist = line.split(': ')[1].strip(' \n')
        if title is not None and artist is not None and number is not None:
            with open('{}/salami_chords.txt'.format(number)) as f:
                lines = f.readlines()
                d = float(lines[-1].split()[0])
                p = str(int(round(d%60,0)))
                duration = str(int(d//60))+':'+'0'*(2-len(p))+p
            # and duration is not None:
            ds = ds.append({'title':title,'artist':artist, 'number':number, 'duration':duration}, ignore_index=True)
            title, artist, number, duration = None, None, None, None

In [84]:
ds.head()

,title,artist,number,duration
0,I Don't mind,James Brown,0003,2:31
1,You've got a Friend,Roberta Flack and Donny Hathaway,0004,3:27
2,The Rose,Bette Midler,0006,3:41
3,An Innocent Man,Billy Joel,0010,5:18
4,Lookin' for Love,Johnny Lee,0012,3:32


In [85]:
ds.sort_values(['artist','number'], inplace=True)

# Проверка есть ли одинаковые файлы
### Спойлер - есть

In [5]:
nds = ds
for row in ds.iterrows():
    #print row[1]
    row = row[1]
    same = ds[(ds['title']==row[0]) & (ds['artist']==row[1]) & (ds['number']>row[2])]
    for f in same.iterrows():
        if filecmp.cmp('{}/salami_chords.txt'.format(row[2]),'{}/salami_chords.txt'.format(f[1][2])):
            nds = nds[(nds['title']!=f[1][0]) | (nds['artist']!=f[1][1]) | (nds['number']!=f[1][2])]

In [6]:
for row in nds.iterrows():
    same = nds[(nds['title']==row[1][0]) & (nds['artist']==row[1][1])& (nds['number']!=row[1][2])]
    if len(same)>0:
        print 'row'
        print row
        print 'same'
        print same
        print

row
(397, title           Unchained Melody
artist    The Righteous Brothers
number                      0587
Name: 397, dtype: object)
same
                title                  artist number
686  Unchained Melody  The Righteous Brothers   1007

row
(686, title           Unchained Melody
artist    The Righteous Brothers
number                      1007
Name: 686, dtype: object)
same
                title                  artist number
397  Unchained Melody  The Righteous Brothers   0587



Так как есть строчки, отличающиеся только номером 'number', и соответствующие им файлы также одинаковы, удаляем дупликаты

In [86]:
ds = ds.drop_duplicates(['artist','title'])
ds.index = range(ds.shape[0])

# Сохранение файлов

In [87]:
ds

,title,artist,number,duration
0,Chicago,25 or 6 to 4,0176,4:50
1,Sukiyaki,A Taste Of Honey,1286,3:43
2,Chiquitita,ABBA,0183,5:25
3,"Knowing Me, Knowing You",ABBA,0231,4:04
4,Honey Honey,ABBA,0902,2:54
5,Fernando,Abba,0218,4:16
6,On And On And On,Abba,0894,3:42
7,Take A Chance On Me,Abba,0961,4:02
8,Last Child,Aerosmith,0015,3:26
9,"Oh Me, Oh My (Dreams In My Arms)",Al Green,0573,2:47


In [88]:
ds.to_csv('songs.csv')